In [1]:
import numpy as np
import pandas as pd

## Loading Dataset

In [2]:
dataframe = pd.read_csv('database.csv', na_values=[''])

## Ordering Columns

In [3]:
dataframe.columns.tolist()

['Unnamed: 0',
 'row_close',
 'levelCounter',
 'trial_set',
 'row_start',
 'time_secs',
 'ptcp',
 'feedbackType',
 'correctCounter',
 'Change Flag']

In [4]:
new_order = ['ptcp',
             'trial_set',
             'levelCounter',
             'row_start',
             'row_close',
             'time_secs',
             'feedbackType',
             'correctCounter',
             'Change Flag']

In [5]:
work_df = dataframe[new_order]

## Data Cleaning and Imputation

First we need to watch if after leading all data all conditions remain the same.

In [6]:
table_feedback = work_df.groupby('feedbackType').count()
print(table_feedback)

              ptcp  trial_set  levelCounter  row_start  row_close  time_secs  \
feedbackType                                                                   
congruent      784        784           784        778        784        778   
incongruent    784        784           784        777        784        777   
none           790        790           790        786        790        786   

              correctCounter  Change Flag  
feedbackType                               
congruent                727          727  
incongruent              731          731  
none                     735          735  


In [7]:
print(work_df['ptcp'].unique())
rougth_clean_df = work_df.dropna(how='any')
rougth_clean_df.groupby('feedbackType').count()
print(table_feedback)

['tsvr27' 'tsvr26' 'tsvr25' 'tsvr24' 'tsvr23' 'tsvr22' 'tsvr21' 'tsvr20'
 'tsvr19' 'tsvr18' 'tsvr17' 'tsvr16' 'tsvr15' 'tsvr14' 'tsvr13' 'tsvr12'
 'tsvr11' 'tsvr06' 'tsvr10' 'tsvr09' 'tsvr08' 'tsvr07']
              ptcp  trial_set  levelCounter  row_start  row_close  time_secs  \
feedbackType                                                                   
congruent      784        784           784        778        784        778   
incongruent    784        784           784        777        784        777   
none           790        790           790        786        790        786   

              correctCounter  Change Flag  
feedbackType                               
congruent                727          727  
incongruent              731          731  
none                     735          735  
